In [ ]:
"""
Created on Wed Mar 20 15:07 2024

This script is to make 2D maps of viability limits

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import multimelt.useful_functions as uf

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_data='/bettik/burgardc/DATA/SUMMER_PAPER/interim/'


In [ ]:
geoyear = 2150

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/ElmerIce_'+str(geoyear)+'/'
file_isf_orig = xr.open_dataset(inputpath_mask+'ElmerIce_4km_'+str(geoyear)+'isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
sorted_isf_rignot = [11,69,43,28,12,57,
                     70,44,29,13,58,71,45,30,14,
                     59,72,46,
                     31,
                     15,61,73,47,32,16,48,33,17,62,49,34,18,63,74,
                     50,35,19,64,
                     10,
                     36,20,65,51,37,
                     22,38,52,23,66,53,39,24,
                     67,40,54,75,25,41,
                     26,42,55,68,60,27]
    
file_isf = file_isf_nonnan.sel(Nisf=sorted_isf_rignot)

In [ ]:
isf_mask = file_isf['ISF_mask']

In [ ]:
file_isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')

In [ ]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL'] #'GISS-E2-1-H',

In [ ]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_withoutGISS.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300_withoutGISS.nc')
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br_withoutGISS_ElmerIcegeo'+str(geoyear)+'.nc').sel(model=model_2300)

In [ ]:
mask_basins = xr.open_dataset(inputpath_data + 'Mask_Iceshelf_4km_IMBIE_withNisf.nc')

In [ ]:
GL_fluxes_ref = xr.open_dataset(outputpath_GL + 'all_ref_GL_fluxes_varying_m_ElmerIcegeometries.nc')

In [ ]:
GL_flux_ag = GL_fluxes_ref['flux_Gt_ref'].sel(geoyear=geoyear)

PREPARE DATA

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

sens_weights = sens_weights.drop_sel(model='GISS-E2-1-H')

In [ ]:
bay_weights_2300 = weight_2300_file['bay_weights']

In [ ]:
# CALVING = 0
mass_balance_weighted_yy_2300 = (file_viability_info['MASS_BALANCE'] - file_viability_info['CALVING']).sel(time=range(geoyear,2300)).sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
weighted_quantiles_2300 = mass_balance_weighted_yy_2300.quantile([0,0.1,0.33,0.66,0.5,0.9,1],dim=['model','param','m'])

In [ ]:
unviable_times_2300 = weighted_quantiles_2300 > 0

In [ ]:
viability_diff_2300 = unviable_times_2300.sel(time=range(2298,geoyear-1,-1)).cumsum('time').diff('time')
limit_max_2300 = viability_diff_2300.time.where(viability_diff_2300 == 0).max('time') + 1
limit_max_2300 = limit_max_2300.where(limit_max_2300<2299)
limit_max_2300 = limit_max_2300.where(~(np.isnan(limit_max_2300) & unviable_times_2300.sel(time=geoyear)), geoyear)

limit_max_full_2300 = limit_max_2300.where(np.isfinite(limit_max_2300), 2305)

unviable_isf_2300 = (unviable_times_2300.time >= limit_max_full_2300)
count_unviable_isf_2300 = (unviable_times_2300.time >= limit_max_full_2300).sum('Nisf')


In [ ]:
likelihood_list = ['vunlikely','unlikely','about','likely','vlikely']

In [ ]:
quant_distrib_2300 = unviable_isf_2300.sel(quantile=[0,0.1,0.33,0.66,0.9,1]).sum('quantile')

PREPARE PLOT

In [ ]:
grounded_msk03 = file_isf['ground_mask'].where(file_isf['ground_mask']==0,3)
grounded_msk = (grounded_msk03.where(grounded_msk03!=3,1)-1)*-1

In [ ]:
grounded_basin = mask_basins['Iceshelf'].where(grounded_msk == 0)

In [ ]:
icesheet_msk_0inf = file_isf_mask.where(file_isf_mask!=1,0)
icesheet_msk = icesheet_msk_0inf.where(icesheet_msk_0inf < 1, 1)

In [ ]:
### Colorbar:
cmap = mpl.cm.YlOrRd # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
# cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap_new = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

alpha = 0.8
cmap_with_alpha = cmap(np.arange(cmap_new.N))
cmap_with_alpha[:, -1] = alpha
cmap_new2 = mpl.colors.ListedColormap(cmap_with_alpha)

# define the bins and normalize
bounds = np.arange(6)
norm = mpl.colors.BoundaryNorm(bounds, cmap_new2.N)




In [ ]:
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = mpl.colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

In [ ]:
yy = 2297
via_to_plot = quant_distrib_2300.sel(time=yy) 
#via_to_plot = quant_distrib.sel(time=yy,scenario=scen)

if geoyear == 2100:
    weird_isf = [57,47,32,36,41]
elif geoyear == 2150:
    weird_isf = [57,72,35,19,36,41,26]

map_to_plot = file_isf['ISF_mask'] * np.nan
map_to_plot2 = file_isf['ISF_mask'] * np.nan
map_to_plot3 = file_isf['ISF_mask'] * np.nan
map_to_plot4 = file_isf['ISF_mask'] * np.nan

grounded_basin_kisf = file_isf['ISF_mask'] * np.nan

for kisf in tqdm(file_isf['Nisf']): #
    map_to_plot = map_to_plot.where(file_isf['ISF_mask'] != kisf, via_to_plot.sel(Nisf=kisf))
    other_Nisf = mask_basins['ID_IMBIE'].sel(Nisf=kisf)
    #print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values, other_Nisf.values)
    grounded_basin_kisf = grounded_basin_kisf.where(grounded_basin != other_Nisf, kisf)
    map_to_plot2 = map_to_plot2.where(grounded_basin != other_Nisf, via_to_plot.sel(Nisf=kisf))
    if kisf.values in weird_isf:
        map_to_plot3 = map_to_plot3.where(grounded_basin != other_Nisf, 1.0)
        map_to_plot4 = map_to_plot4.where(file_isf['ISF_mask'] != kisf, 1.0)
    else:
        map_to_plot3 = map_to_plot3.where(grounded_basin != other_Nisf, np.nan)
        map_to_plot4 = map_to_plot4.where(file_isf['ISF_mask'] != kisf, np.nan)

map_to_plot_all = map_to_plot.combine_first(map_to_plot2)
grounded_basin_kisf = grounded_basin_kisf.where(np.isfinite(grounded_basin_kisf), 76)

fig = plt.figure(figsize=(8.25,8.25))

# MAR masks:
grd=file_isf['ISF_mask']
msk_ice = grd.where( (grd>0.) )* 0 + 1

# Basin masks:
basin=xr.open_dataset('/bettik/burgardc/DATA/SUMMER_PAPER/interim/Mask_Iceshelf_4km_IMBIE_withNisf.nc')

##########################################################################
# PLOT :

#----------
# Colorbar:
#cbar_range = np.arange(0.,4200.,200.)
#newcolors = col(np.linspace(0.47, 1.00,256))

#----------
# Defining colormap:

# moving the zero of colorbar
# NB: modify the Ncool to Nwarm ratio (total=256) to place zero as desired.
#Ncool=int(256*(-np.amin(cbar_range)/(np.amax(cbar_range)-np.amin(cbar_range))))
#Nwarm=256-Ncool
#col = cm.get_cmap('PuOr', 256)
#tmp1 = col(np.linspace(0.47, 1.00, Ncool)) # decrease first number to have more white in the middle light-blue colors
#tmp2 = col(np.linspace(0.00, 0.51, Nwarm)) # increase second number to have more white in the middle light-yellow colors
#newcolors = np.append(tmp1[::-1,:],tmp2[::-1,:],axis=0)
#newcolors = col(np.linspace(0.47, 1.00,256))
#cmap_new = ListedColormap(newcolors)

levels = range(10,77)

#cax=fig.add_axes([0.52, 0.02, 0.43, 0.015]) # color bar
im0=plt.contourf(grd.x,grd.y,map_to_plot,extend='max',cmap=cmap_new2, norm=norm, levels=bounds)
plt.contourf(grd.x,grd.y,map_to_plot2,extend='max',cmap=cmap_new2, norm=norm, levels=bounds) #, alpha=0.7

plt.contourf(grd.x,grd.y,map_to_plot3,cmap=truncate_colormap(mpl.cm.Greys, 0.6, 0.8))
plt.contourf(grd.x,grd.y,map_to_plot4,cmap=truncate_colormap(mpl.cm.Greys, 0.6, 0.8))

#plt.contour(grd.x,grd.y,map_to_plot2,extend='max',colors='grey',zorder=15)
plt.contour(grd.x,grd.y,file_isf['ISF_mask'],levels,linewidths=0.7,colors='darkgrey',linestyles='solid',zorder=10) #.where(np.isnan(grounded_msk))
plt.contour(grd.x,grd.y,grounded_basin_kisf,levels,linewidths=0.7,colors='darkgrey',linestyles='solid',zorder=10)
plt.contour(grd.x,grd.y,grounded_msk,linewidths=0.7,colors='darkgrey',zorder=12)
plt.contour(grd.x,grd.y,icesheet_msk,linewidths=0.9,colors='black',zorder=15)

plt.title('Viability probability '+str(yy)+' Elmergeo '+str(geoyear))

ax2 = fig.add_axes([0.90, 0.1, 0.03, 0.8])
cb = mpl.colorbar.ColorbarBase(ax2, cmap=cmap_new2, norm=norm,
    spacing='proportional', ticks=np.arange(0.5,5), boundaries=bounds) #, format='%1i'
cb.set_ticklabels(likelihood_list, rotation=90)

#plt.tight_layout()
fig.savefig(plot_path + 'map_2D_likelihood_'+str(yy)+'_mod2300_withoutGISS_ElmerIcegeo'+str(geoyear)+'.png', dpi=300)

ANALYTICS AREA PROPORTION

In [ ]:
map_lim = [-3000000,3000000]


file_mask_orig = xr.open_dataset(inputpath_data+'BedMachine_v2_aggregated4km_allvars.nc')
file_mask_orig_cut = uf.cut_domain_stereo(file_mask_orig, map_lim, map_lim)

ice_thickness = file_mask_orig_cut['thickness']

In [ ]:
grid_cell_area_file = xr.open_dataset(inputpath_data+'gridarea_ISMIP6_AIS_4000m_grid.nc').sel(x=file_isf.x,y=file_isf.y)
true_grid_cell_area = grid_cell_area_file['cell_area'].drop('lon').drop('lat')

In [ ]:
total_grounded_area = (np.isfinite(grounded_basin).astype(int) * true_grid_cell_area).sum()
total_grounded_volume = (np.isfinite(grounded_basin).astype(int) * true_grid_cell_area * ice_thickness).sum()

In [ ]:
file_mask_orig['thickness'].plot()

In [ ]:
vol_kisf_list = []
for kisf in file_isf.Nisf:
    grounded_vol_kisf = ((grounded_basin == kisf).astype(int) * true_grid_cell_area * ice_thickness).sum()
    vol_kisf_list.append(grounded_vol_kisf.assign_coords({'Nisf': kisf}))
grounded_vol_allisf = xr.concat(vol_kisf_list, dim='Nisf')

In [ ]:
GL_flux = file_viability_info['ABUMIP'].isel(scenario=0,model=0,param=0,time=0)

In [ ]:
limit_int = limit_max_full_2300.sel(scenario='ssp585', quantile=0.33)

In [ ]:
time_to_empty = (grounded_vol_allisf / GL_flux.mean('m'))

In [ ]:
plt.scatter(limit_int,time_to_empty)

In [ ]:
for kisf in file_isf.Nisf:
    print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
mask_below_sea_level = (file_mask_orig['bed'] < 0) & (grounded_msk == 0)
sea_level_eq_volume = (file_mask_orig['surface'].where(mask_below_sea_level) * true_grid_cell_area)

In [ ]:
scen = 'ssp585'
yy = 2297
via_to_plot = quant_distrib_2300.sel(time=yy,scenario=scen) 
#via_to_plot = quant_distrib.sel(time=yy,scenario=scen)

map_to_plot = file_isf['ISF_mask'] * np.nan
map_to_plot2 = file_isf['ISF_mask'] * np.nan
grounded_basin_kisf = file_isf['ISF_mask'] * np.nan

for kisf in tqdm(file_isf['Nisf']): #
    map_to_plot = map_to_plot.where(file_isf['ISF_mask'] != kisf, via_to_plot.sel(Nisf=kisf))
    other_Nisf = mask_basins['ID_IMBIE'].sel(Nisf=kisf)
    #print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values, other_Nisf.values)
    grounded_basin_kisf = grounded_basin_kisf.where(grounded_basin != other_Nisf, kisf)
    map_to_plot2 = map_to_plot2.where(grounded_basin != other_Nisf, via_to_plot.sel(Nisf=kisf))
map_to_plot_all = map_to_plot.combine_first(map_to_plot2)
grounded_basin_kisf = grounded_basin_kisf.where(np.isfinite(grounded_basin_kisf), 76)

In [ ]:
((map_to_plot2 > 3) * sea_level_eq_volume).plot()

In [ ]:
((map_to_plot2 > 3) * sea_level_eq_volume * 917./1028.).sum() / (3.625e14)

In [ ]:
map_to_plot2_2200 = map_to_plot2.copy()

In [ ]:
((map_to_plot2 > 3).astype(int) - (map_to_plot2_2200 > 3).astype(int)).plot()

In [ ]:
plt.figure()
(map_to_plot2_2200 > 3).plot()

In [ ]:
mask_below_sea_level.plot()

In [ ]:
(mask_below_sea_level * (map_to_plot2_2200 > 3) * true_grid_cell_area).sum() / ((map_to_plot2_2200 > 3) * true_grid_cell_area).sum()

In [ ]:
# PERCENTAGE OF AREA WHERE BED IS BELOW SEA LEVEL
(mask_below_sea_level * (map_to_plot2 > 3) * true_grid_cell_area).sum() / ((map_to_plot2 > 3) * true_grid_cell_area).sum()

In [ ]:
((map_to_plot2 > 3) * sea_level_eq_volume).sum() 

In [ ]:
((map_to_plot2 > 3) * true_grid_cell_area).sum() / total_grounded_area

In [ ]:
((map_to_plot2 > 3) * true_grid_cell_area * ice_thickness).sum() / total_grounded_volume

PIE CHARTS

In [ ]:
dx

In [ ]:
kisf_mask.x

In [ ]:
center_x

In [ ]:
center_x_list = []
center_y_list = []
for kisf in tqdm(file_isf.Nisf):
    kisf_mask = isf_mask.where(isf_mask == kisf, drop=True)
    center_x = kisf_mask.x.isel(x=np.ceil(len(kisf_mask.x)/2).astype(int))
    center_y = kisf_mask.y.isel(y=np.ceil(len(kisf_mask.y)/2).astype(int))

    dx = kisf_mask.x.isel(x=1) - kisf_mask.x.isel(x=0)

    if np.isnan(kisf_mask.sel(x=center_x,y=center_y)):
        center_x = kisf_mask.x.min() 
        while np.isnan(kisf_mask.sel(x=center_x,y=center_y)):
            center_x = center_x + dx
            if center_x.values in kisf_mask.x.values: 
                center_x = center_x
            else:
                center_x = center_x + dx

    center_x_list.append(xr.DataArray(data=center_x.values).assign_coords({'Nisf': kisf}))
    center_y_list.append(xr.DataArray(data=center_y.values).assign_coords({'Nisf': kisf}))

center_dim = xr.Dataset()
center_dim['center_x'] = xr.concat(center_x_list, dim='Nisf')
center_dim['center_y'] = xr.concat(center_y_list, dim='Nisf')


In [ ]:
ocean_basin = mask_basins['Iceshelf_extrap'].where((grounded_msk == 1) & (file_isf_mask == 0))

In [ ]:
center_x_oc_list = []
center_y_oc_list = []
for kisf in tqdm(file_isf.Nisf):
    other_kisf = mask_basins['ID_IMBIE'].sel(Nisf=kisf)

    if kisf == 67:
        center_x_oc = xr.DataArray(data=[-1.825e6])
        center_y_oc = xr.DataArray(data=[2e5])
    else:
        oc_mask = isf_mask.where(ocean_basin == other_kisf, drop=True)
        if len(oc_mask.x) > 1:
            center_x_oc = oc_mask.x.isel(x=np.ceil(len(oc_mask.x)/2).astype(int))
            dx = oc_mask.x.isel(x=1) - oc_mask.x.isel(x=0)
        else:
            center_x_oc = oc_mask.x[0].astype(int)
        if len(oc_mask.y) > 1:
            center_y_oc = oc_mask.y.isel(y=np.ceil(len(oc_mask.y)/2).astype(int))
        else:
            center_y_oc = oc_mask.y[0].astype(int)
    
    # individual corrections
    if kisf == 11:
        center_x_oc = xr.DataArray(data=[-1.97e6])
        center_y_oc = xr.DataArray(data=[2.61e6])
    elif kisf == 10:
        center_x_oc = xr.DataArray(data=[-2.44e5])
        center_y_oc = xr.DataArray(data=[-2.6e6])
    elif kisf == 43:
        center_x_oc = xr.DataArray(data=[-7.8e5])
        center_y_oc = xr.DataArray(data=[2.0e6])
    elif kisf == 44:
        center_x_oc = xr.DataArray(data=[9.4e4])
    elif kisf == 45:
        center_x_oc = xr.DataArray(data=[1.35e6])
        center_y_oc = xr.DataArray(data=[2.5e6])
    elif kisf == 46:
        center_x_oc = xr.DataArray(data=[2.27e6])
        center_y_oc = xr.DataArray(data=[1.4e6])        
    elif kisf == 59:
        center_x_oc = xr.DataArray(data=[2.01e6])
        center_y_oc = xr.DataArray(data=[2.06e6])  
    elif kisf == 14:
        center_x_oc = xr.DataArray(data=[1.7e6])
        center_y_oc = xr.DataArray(data=[1.9e6])
    elif kisf == 31:
        center_x_oc = xr.DataArray(data=[2.8e6])
        center_y_oc = xr.DataArray(data=[1.1e6])
    elif kisf == 73:
        center_x_oc = xr.DataArray(data=[2.9e6])
    elif kisf == 63:
        center_y_oc = xr.DataArray(data=[-2.38e6])
    elif kisf == 50:
        center_x_oc = xr.DataArray(data=[1.5e5])
        center_y_oc = xr.DataArray(data=[-1.96e6])
    elif kisf == 72:
        center_x_oc = xr.DataArray(data=[2.26e6])
        center_y_oc = xr.DataArray(data=[1.94e6])
    elif kisf == 74:
        center_x_oc = xr.DataArray(data=[4.03e5])
        center_y_oc = xr.DataArray(data=[-2.18e6])
    elif kisf == 34:
        center_x_oc = xr.DataArray(data=[1.29e6])
        center_y_oc = xr.DataArray(data=[-2.33e6])
    elif kisf == 18:
        center_x_oc = xr.DataArray(data=[9.8e5])
        center_y_oc = xr.DataArray(data=[-2.27e6])
    elif kisf == 62:
        center_x_oc = xr.DataArray(data=[1.8e6])
        center_y_oc = xr.DataArray(data=[-2.24e6])
    elif kisf == 17:
        center_x_oc = xr.DataArray(data=[2.07e6])
        center_y_oc = xr.DataArray(data=[-1.95e6])
    elif kisf == 33:
        #center_x_oc = xr.DataArray(data=[2.07e6])
        center_y_oc = xr.DataArray(data=[-2.03e6])
    elif kisf == 48:
        center_x_oc = xr.DataArray(data=[2.7e6])
        center_y_oc = xr.DataArray(data=[-1.3e6])
    elif kisf == 15:
        center_x_oc = xr.DataArray(data=[2.36e6])
        #center_y_oc = xr.DataArray(data=[-2.03e6])
    elif kisf == 16:
        center_x_oc = xr.DataArray(data=[2.6e6])
        #center_y_oc = xr.DataArray(data=[-2.03e6])
    elif kisf == 32:
        #center_x_oc = xr.DataArray(data=[2.6e6])
        center_y_oc = xr.DataArray(data=[-6.6e5])
    elif kisf == 64:
        center_x_oc = xr.DataArray(data=[6.0e4])
        center_y_oc = xr.DataArray(data=[-1.48e6])
    elif kisf == 35:
        center_x_oc = xr.DataArray(data=[2.34e5])
        center_y_oc = xr.DataArray(data=[-1.7e6])
    elif kisf == 19:
        center_x_oc = xr.DataArray(data=[3.56e5])
        center_y_oc = xr.DataArray(data=[-1.58e6])
    elif kisf == 65:
        center_x_oc = xr.DataArray(data=[-8.3e5])
        center_y_oc = xr.DataArray(data=[-1.56e6])
    elif kisf == 20:
        #center_x_oc = xr.DataArray(data=[-8.3e5])
        center_y_oc = xr.DataArray(data=[-1.4e6])
    elif kisf == 37:
        center_x_oc = xr.DataArray(data=[-1.1e6])
        center_y_oc = xr.DataArray(data=[-1.35e6])
    elif kisf == 22:
        center_x_oc = xr.DataArray(data=[-1.3e6])
        center_y_oc = xr.DataArray(data=[-1.67e6])
    elif kisf == 52:
        center_x_oc = xr.DataArray(data=[-1.65e6])
        center_y_oc = xr.DataArray(data=[-2.22e6])
    elif kisf == 23:
        center_x_oc = xr.DataArray(data=[-2.15e6])
        center_y_oc = xr.DataArray(data=[-1.11e6])
    elif kisf == 39:
        center_x_oc = xr.DataArray(data=[-2.3e6])
        center_y_oc = xr.DataArray(data=[-1.6e5])
    elif kisf == 66:
        center_x_oc = xr.DataArray(data=[-2.7e6])
        center_y_oc = xr.DataArray(data=[-1.3e6])
    elif kisf == 38:
        center_x_oc = xr.DataArray(data=[-1.71e6])
        center_y_oc = xr.DataArray(data=[-1.07e6])
    elif kisf == 53:
        center_x_oc = xr.DataArray(data=[-2.42e6])
        center_y_oc = xr.DataArray(data=[-6.95e5])
    elif kisf == 41:
        center_x_oc = xr.DataArray(data=[-2.65e6])
        #center_y_oc = xr.DataArray(data=[-6.95e5])
    elif kisf == 75:
        center_x_oc = xr.DataArray(data=[-2.70e6])
        center_y_oc = xr.DataArray(data=[5.7e5])
    elif kisf == 24:
        center_x_oc = xr.DataArray(data=[-2.04e6])
        center_y_oc = xr.DataArray(data=[0])
    elif kisf == 67:
        center_x_oc = xr.DataArray(data=[-1.9e6])
        #center_y_oc = xr.DataArray(data=[0])
    elif kisf == 40:
        center_x_oc = xr.DataArray(data=[-2.11e6])
        center_y_oc = xr.DataArray(data=[2.12e5])
    elif kisf == 54:
        center_x_oc = xr.DataArray(data=[-2.33e6])
        center_y_oc = xr.DataArray(data=[4.8e5])
    elif kisf == 25:
        center_x_oc = xr.DataArray(data=[-2.66e6])
        center_y_oc = xr.DataArray(data=[2.3e5])
    elif kisf == 26:
        #center_x_oc = xr.DataArray(data=[-2.30e6])
        center_y_oc = xr.DataArray(data=[1.46e6])
    elif kisf == 42:
        center_x_oc = xr.DataArray(data=[-2.16e6])
        #center_y_oc = xr.DataArray(data=[1.46e6])
    elif kisf == 68:
        center_x_oc = xr.DataArray(data=[-1.69e6])
        center_y_oc = xr.DataArray(data=[1.08e6])
    elif kisf == 27:
        center_x_oc = xr.DataArray(data=[-1.37e6])
        center_y_oc = xr.DataArray(data=[8.75e5])
        
    center_x_oc_list.append(xr.DataArray(data=center_x_oc.values).assign_coords({'Nisf': kisf}))
    center_y_oc_list.append(xr.DataArray(data=center_y_oc.values).assign_coords({'Nisf': kisf}))

#center_dim = xr.Dataset()
center_dim['center_x_oc'] = xr.concat(center_x_oc_list, dim='Nisf')
center_dim['center_y_oc'] = xr.concat(center_y_oc_list, dim='Nisf')


In [ ]:
flux_ref_br, _ = xr.broadcast(GL_flux_ag, file_viability_info['ABUMIP'])

In [ ]:
flux_ref_weighted_mean = flux_ref_br.weighted(weight_2300_file['bay_weights']).mean(['param','model','m','time'])
ABUMIP_weighted_mean = file_viability_info['ABUMIP'].weighted(weight_2300_file['bay_weights']).mean(['param','model','m','time',])

In [ ]:

yy = 2297
via_to_plot = quant_distrib_2300.sel(time=yy)
#via_to_plot = quant_distrib.sel(time=yy,scenario=scen)

map_to_plot = file_isf['ISF_mask'] * np.nan
map_to_plot2 = file_isf['ISF_mask'] * np.nan
grounded_basin_kisf = file_isf['ISF_mask'] * np.nan

for kisf in tqdm(file_isf['Nisf']):
    map_to_plot = map_to_plot.where(file_isf['ISF_mask'] != kisf, via_to_plot.sel(Nisf=kisf))
    other_Nisf = mask_basins['ID_IMBIE'].sel(Nisf=kisf)
    #print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values, other_Nisf.values)
    grounded_basin_kisf = grounded_basin_kisf.where(grounded_basin != other_Nisf, kisf)
    map_to_plot2 = map_to_plot2.where(grounded_basin != other_Nisf, via_to_plot.sel(Nisf=kisf))
map_to_plot_all = map_to_plot.combine_first(map_to_plot2)
grounded_basin_kisf = grounded_basin_kisf.where(np.isfinite(grounded_basin_kisf), 76)

fig, ax = plt.subplots(figsize=(8.25,8.25))
# MAR masks:
grd=file_isf['ISF_mask']
msk_ice = grd.where( (grd>0.) )* 0 + 1

# Basin masks:
basin=xr.open_dataset('/bettik/burgardc/DATA/SUMMER_PAPER/interim/Mask_Iceshelf_4km_IMBIE_withNisf.nc')

##########################################################################
# PLOT :

#----------
# Colorbar:
#cbar_range = np.arange(0.,4200.,200.)
#newcolors = col(np.linspace(0.47, 1.00,256))

#----------
# Defining colormap:

# moving the zero of colorbar
# NB: modify the Ncool to Nwarm ratio (total=256) to place zero as desired.
#Ncool=int(256*(-np.amin(cbar_range)/(np.amax(cbar_range)-np.amin(cbar_range))))
#Nwarm=256-Ncool
#col = cm.get_cmap('PuOr', 256)
#tmp1 = col(np.linspace(0.47, 1.00, Ncool)) # decrease first number to have more white in the middle light-blue colors
#tmp2 = col(np.linspace(0.00, 0.51, Nwarm)) # increase second number to have more white in the middle light-yellow colors
#newcolors = np.append(tmp1[::-1,:],tmp2[::-1,:],axis=0)
#newcolors = col(np.linspace(0.47, 1.00,256))
#cmap_new = ListedColormap(newcolors)

levels = range(10,77)

#cax=fig.add_axes([0.52, 0.02, 0.43, 0.015]) # color bar
im0=plt.contourf(grd.x,grd.y,map_to_plot,extend='max',cmap=cmap_new, norm=norm)
ax.contourf(grd.x,grd.y,map_to_plot2,extend='max',cmap=cmap_new, norm=norm, alpha=0.7)
#plt.contour(grd.x,grd.y,map_to_plot2,extend='max',colors='grey',zorder=15)
ax.contour(grd.x,grd.y,grounded_basin_kisf,levels,linewidths=0.5,colors='grey',zorder=10)
ax.contour(grd.x,grd.y,grounded_msk,linewidths=0.5,colors='black',zorder=10)
ax.contour(grd.x,grd.y,icesheet_msk,linewidths=0.1,colors='black',zorder=15)
ax.plot([center_dim['center_x'].squeeze().values, center_dim['center_x_oc'].squeeze().values], [center_dim['center_y'].squeeze().values, center_dim['center_y_oc'].squeeze().values], linestyle="-", color = 'k', linewidth = 0.5)



for kisf in tqdm(file_isf['Nisf']):
    if ((ABUMIP_weighted_mean.sel(Nisf=kisf) > 0) & ((ABUMIP_weighted_mean.sel(Nisf=kisf) - flux_ref_weighted_mean.sel(Nisf=kisf)) > 0)):
        pos_x = center_dim['center_x_oc'].sel(Nisf=kisf).squeeze().values
        pos_y = center_dim['center_y_oc'].sel(Nisf=kisf).squeeze().values
        size = [((ABUMIP_weighted_mean.sel(Nisf=kisf) - flux_ref_weighted_mean.sel(Nisf=kisf))/ABUMIP_weighted_mean.sel(Nisf=kisf)).values,(flux_ref_weighted_mean.sel(Nisf=kisf)/ABUMIP_weighted_mean.sel(Nisf=kisf)).values]
        radius = ABUMIP_weighted_mean.sel(Nisf=kisf) /ABUMIP_weighted_mean.max().values + 0.2
        
        ax_sub= inset_axes(ax, width=radius.values.item(), height=radius.values.item(), loc=10, 
                           bbox_to_anchor=(pos_x, pos_y),
                           bbox_transform=ax.transData, 
                           borderpad=0)
        ax_sub.pie(np.array(size), colors=['steelblue','lightblue'], startangle=90)
        ax_sub.set_aspect("equal")
    else:
        print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values)

    

ax.set_title('Viability probability '+str(yy)+' ElmerIcegeo '+str(geoyear))

ax2 = fig.add_axes([0.90, 0.1, 0.03, 0.8])
cb = mpl.colorbar.ColorbarBase(ax2, cmap=cmap_new, norm=norm,
    spacing='proportional', ticks=np.arange(0.5,5), boundaries=bounds) #, format='%1i'
cb.set_ticklabels(likelihood_list, rotation=90)

#plt.tight_layout()
fig.savefig(plot_path + 'map_2D_likelihood_'+str(yy)+'_withpiecharts_ElmerIcegeo'+str(geoyear)+'.png', dpi=300)

In [ ]:
((ABUMIP_weighted_mean.sel(Nisf=kisf) - flux_ref_weighted_mean.sel(Nisf=kisf)) > 0)

In [ ]:
file_isf['isf_name'].sel(Nisf=kisf).load()

In [ ]:
file_viability_info['ABUMIP'].sel(Nisf=kisf).weighted(weight_file['bay_weights']

In [ ]:
ABUMIP_weighted_mean.sel(Nisf=kisf)

In [ ]:
flux_ref_weighted_mean.sel(Nisf=kisf)

In [ ]:
flux_ref_weighted_mean

In [ ]:
scen = 'ssp126'
yy = 2100
via_to_plot = quant_distrib_2300.sel(time=yy,scenario=scen)
#via_to_plot = quant_distrib.sel(time=yy,scenario=scen)

map_to_plot = file_isf['ISF_mask'] * np.nan
map_to_plot2 = file_isf['ISF_mask'] * np.nan
grounded_basin_kisf = file_isf['ISF_mask'] * np.nan

for kisf in tqdm(file_isf['Nisf']):
    map_to_plot = map_to_plot.where(file_isf['ISF_mask'] != kisf, via_to_plot.sel(Nisf=kisf))
    other_Nisf = mask_basins['ID_IMBIE'].sel(Nisf=kisf)
    #print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values, other_Nisf.values)
    grounded_basin_kisf = grounded_basin_kisf.where(grounded_basin != other_Nisf, kisf)
    map_to_plot2 = map_to_plot2.where(grounded_basin != other_Nisf, via_to_plot.sel(Nisf=kisf))
map_to_plot_all = map_to_plot.combine_first(map_to_plot2)
grounded_basin_kisf = grounded_basin_kisf.where(np.isfinite(grounded_basin_kisf), 76)


fig, ax = plt.subplots(figsize=(8.25,8.25))
# MAR masks:
grd=file_isf['ISF_mask']
msk_ice = grd.where( (grd>0.) )* 0 + 1

# Basin masks:
basin=xr.open_dataset('/bettik/burgardc/DATA/SUMMER_PAPER/interim/Mask_Iceshelf_4km_IMBIE_withNisf.nc')

##########################################################################
# PLOT :

#----------
# Colorbar:
#cbar_range = np.arange(0.,4200.,200.)
#newcolors = col(np.linspace(0.47, 1.00,256))

#----------
# Defining colormap:

# moving the zero of colorbar
# NB: modify the Ncool to Nwarm ratio (total=256) to place zero as desired.
#Ncool=int(256*(-np.amin(cbar_range)/(np.amax(cbar_range)-np.amin(cbar_range))))
#Nwarm=256-Ncool
#col = cm.get_cmap('PuOr', 256)
#tmp1 = col(np.linspace(0.47, 1.00, Ncool)) # decrease first number to have more white in the middle light-blue colors
#tmp2 = col(np.linspace(0.00, 0.51, Nwarm)) # increase second number to have more white in the middle light-yellow colors
#newcolors = np.append(tmp1[::-1,:],tmp2[::-1,:],axis=0)
#newcolors = col(np.linspace(0.47, 1.00,256))
#cmap_new = ListedColormap(newcolors)

levels = range(10,77)

#cax=fig.add_axes([0.52, 0.02, 0.43, 0.015]) # color bar
im0=plt.contourf(grd.x,grd.y,map_to_plot,extend='max',cmap=cmap_new, norm=norm, levels=bounds)
plt.contourf(grd.x,grd.y,map_to_plot2,extend='max',cmap=cmap_new, norm=norm, alpha=0.7, levels=bounds)
#plt.contour(grd.x,grd.y,map_to_plot2,extend='max',colors='grey',zorder=15)
ax.contour(grd.x,grd.y,grounded_basin_kisf,levels,linewidths=0.5,colors='grey',zorder=0)
ax.contour(grd.x,grd.y,grounded_msk,linewidths=0.5,colors='black',zorder=0)
ax.contour(grd.x,grd.y,icesheet_msk,linewidths=0.1,colors='black',zorder=5)
ax.plot([center_dim['center_x'].squeeze().values, center_dim['center_x_oc'].squeeze().values], [center_dim['center_y'].squeeze().values, center_dim['center_y_oc'].squeeze().values], linestyle="-", color = 'k', linewidth = 0.5)

for kisf in tqdm(file_isf['Nisf']):
    pos_x = center_dim['center_x_oc'].sel(Nisf=kisf).squeeze().values
    pos_y = center_dim['center_y_oc'].sel(Nisf=kisf).squeeze().values
    size = [((ABUMIP_weighted_mean.sel(Nisf=kisf) - flux_ref_weighted_mean.sel(Nisf=kisf))/ABUMIP_weighted_mean.sel(Nisf=kisf)).values,(flux_ref_weighted_mean.sel(Nisf=kisf)/ABUMIP_weighted_mean.sel(Nisf=kisf)).values]
    prop_ABUMIP = ABUMIP_weighted_mean.sel(Nisf=kisf) / ABUMIP_weighted_mean.max().values
    prop_ref = flux_ref_weighted_mean.sel(Nisf=kisf) / ABUMIP_weighted_mean.max().values
    radius = np.sqrt(prop_ABUMIP/np.pi) * 3
    radius2 = np.sqrt(prop_ref/np.pi) * 3
    
    ax_sub = inset_axes(ax, width=radius.values.item(), height=radius.values.item(), loc=10, 
                       bbox_to_anchor=(pos_x, pos_y),
                       bbox_transform=ax.transData, 
                       borderpad=0)
    ax_sub.pie(np.array(size), colors=['steelblue','lightblue'], startangle=90)
    #ax_sub.set_aspect("equal")

    ax_sub2 = inset_axes(ax, width=radius2.values.item(), height=radius2.values.item(), loc=10, 
                   bbox_to_anchor=(pos_x, pos_y),
                   bbox_transform=ax.transData, 
                   borderpad=0)
    ax_sub2.pie(np.array([1]), colors=['lightskyblue'], startangle=90)

    ax_sub2.set_aspect("equal")

#ax2 = ax.twinx()
#ax2.scatter(center_dim['center_x_oc'], center_dim['center_y_oc'], s=flux_ref_weighted_mean, color='lightskyblue', zorder=20)

ax.set_title('Viability probability '+str(yy)+' '+scen)

ax2 = fig.add_axes([0.90, 0.1, 0.03, 0.8])
cb = mpl.colorbar.ColorbarBase(ax2, cmap=cmap_new, norm=norm,
    spacing='proportional', ticks=np.arange(0.5,5), boundaries=bounds) #, format='%1i'
cb.set_ticklabels(likelihood_list, rotation=90)

#plt.tight_layout()
fig.savefig(plot_path + 'map_2D_likelihood_'+str(yy)+'_'+scen+'_withpiecharts_withproportionality.png', dpi=300)

Compute percentage

2300 values

In [ ]:
# CALVING = 0
non_viable_ensmembers_2300 = (file_viability_info['MASS_BALANCE'] - file_viability_info['CALVING']).squeeze().drop('scenario').sel(model=model_2300) > 0

non_via_diff_2300 = non_viable_ensmembers_2300.sel(time=range(2299,geoyear-1,-1)).cumsum('time').diff('time')
limit_via_max_2300 = non_via_diff_2300.time.where(non_via_diff_2300 == 0).max('time') + 1
limit_via_max_2300 = limit_via_max_2300.where(limit_via_max_2300<2299)
limit_via_max_2300 = limit_via_max_2300.where(~(np.isnan(limit_via_max_2300) & non_viable_ensmembers_2300.sel(time=geoyear)), geoyear)

In [ ]:
# CALVING = 0
time_br_2300, _ = xr.broadcast(file_viability_info.time, (file_viability_info['MASS_BALANCE'] - file_viability_info['CALVING']).sel(model=model_2300).squeeze().drop('scenario'))

In [ ]:
non_viable_time_2300 = limit_via_max_2300

In [ ]:
nonvia_GLflux_2300 = file_viability_info['ABUMIP'].sel(model=model_2300).where(time_br_2300 == non_viable_time_2300, drop=True).max('time')
nonvia_SMB_2300 = file_viability_info['SMB'].squeeze().drop('scenario').sel(model=model_2300).where(time_br_2300 == non_viable_time_2300, drop=True).max('time') * (-1)
nonvia_BMB_2300 = file_viability_info['BMB'].sel(model=model_2300).where(time_br_2300 == non_viable_time_2300, drop=True).max('time')
nonvia_calv_2300 = 0 * file_viability_info['CALVING'].sel(model=model_2300).where(time_br_2300 == non_viable_time_2300, drop=True).max('time')

In [ ]:
nonvia_SMB_perc_2300 = nonvia_SMB_2300 / nonvia_GLflux_2300 * 100
nonvia_BMB_perc_2300 = nonvia_BMB_2300 / nonvia_GLflux_2300 * 100
calv_perc_2300 = nonvia_calv_2300 / nonvia_GLflux_2300 * 100

In [ ]:
nonvia_SMB_perc_mean_2300 = nonvia_SMB_perc_2300.weighted(bay_weights * sens_weights.sel(model=model_2300)).mean(['param','model','m'])
nonvia_BMB_perc_mean_2300 = nonvia_BMB_perc_2300.weighted(bay_weights * sens_weights.sel(model=model_2300)).mean(['param','model','m'])
calv_perc_mean_2300 = calv_perc_2300.weighted(bay_weights * sens_weights.sel(model=model_2300)).mean(['param','model','m'])

In [ ]:
plt.rcParams["hatch.linewidth"] = 2

In [ ]:
np.isfinite(nonvia_BMB_perc_mean_2300.sel(scenario='ssp126')).sum()

In [ ]:
49/53

In [ ]:
(file_isf['ISF_mask'] == 41).plot()

In [ ]:
for kisf in [26,32,34,41,67]:
    print(file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
nonvia_BMB_perc_mean_2300.sel(scenario='ssp585').where(nonvia_BMB_perc_mean_2300.sel(scenario='ssp585') < 50, drop=True)

In [ ]:
nonvia_SMB_perc_mean_2300.sel(scenario='ssp126').mean()

In [ ]:
nonvia_SMB_perc_mean_2300.sel(scenario='ssp585').where(nonvia_SMB_perc_mean_2300.sel(scenario='ssp585') == nonvia_SMB_perc_mean_2300.sel(scenario='ssp585').where(nonvia_SMB_perc_mean_2300.sel(scenario='ssp585') > 0).max(), drop=True)

In [ ]:
36/64

In [ ]:
file_isf['isf_name'].where(file_isf['isf_name'] == 'Crosson', drop=True)

In [ ]:
for kisf in file_isf.Nisf:
    prop_to_reach_ABUMIP = (ABUMIP_weighted_mean.sel(Nisf=kisf) - flux_ref_weighted_mean.sel(Nisf=kisf)) / flux_ref_weighted_mean.sel(Nisf=kisf)
    print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values,np.round(prop_to_reach_ABUMIP*100,1).values)


In [ ]:
plt.scatter(flux_ref_weighted_mean,ABUMIP_weighted_mean)

In [ ]:
(file_isf['ISF_mask'] == 26).plot()

In [ ]:
(nonvia_SMB_perc_mean_2300.sel(scenario='ssp126') > 0).sum()/64

In [ ]:
nonvia_SMB_perc_mean_2300.sel(scenario='ssp585').mean()

In [ ]:
nonvia_SMB_perc_mean_2300

In [ ]:
nonvia_SMB_perc_mean_2300.sel(scenario='ssp126').mean()

In [ ]:
plt.rcParams["hatch.linewidth"] = 2

In [ ]:
#scen = 'ssp126'
yy = 2300

if yy == 2300:
    via_to_plot = quant_distrib_2300.sel(time=2297)
else:
    via_to_plot = quant_distrib.sel(time=yy)

map_to_plot = file_isf['ISF_mask'] * np.nan
map_to_plot2 = file_isf['ISF_mask'] * np.nan
grounded_basin_kisf = file_isf['ISF_mask'] * np.nan

for kisf in tqdm(file_isf['Nisf']):
    map_to_plot = map_to_plot.where(file_isf['ISF_mask'] != kisf, via_to_plot.sel(Nisf=kisf))
    other_Nisf = mask_basins['ID_IMBIE'].sel(Nisf=kisf)
    #print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values, other_Nisf.values)
    grounded_basin_kisf = grounded_basin_kisf.where(grounded_basin != other_Nisf, kisf)
    map_to_plot2 = map_to_plot2.where(grounded_basin != other_Nisf, via_to_plot.sel(Nisf=kisf))
map_to_plot_all = map_to_plot.combine_first(map_to_plot2)
grounded_basin_kisf = grounded_basin_kisf.where(np.isfinite(grounded_basin_kisf), 76)

fig, ax = plt.subplots(figsize=(8.25,8.25))
# MAR masks:
grd=file_isf['ISF_mask']
msk_ice = grd.where( (grd>0.) )* 0 + 1

# Basin masks:
basin=xr.open_dataset('/bettik/burgardc/DATA/SUMMER_PAPER/interim/Mask_Iceshelf_4km_IMBIE_withNisf.nc')

##########################################################################
# PLOT :

#----------
# Colorbar:
#cbar_range = np.arange(0.,4200.,200.)
#newcolors = col(np.linspace(0.47, 1.00,256))

#----------
# Defining colormap:

# moving the zero of colorbar
# NB: modify the Ncool to Nwarm ratio (total=256) to place zero as desired.
#Ncool=int(256*(-np.amin(cbar_range)/(np.amax(cbar_range)-np.amin(cbar_range))))
#Nwarm=256-Ncool
#col = cm.get_cmap('PuOr', 256)
#tmp1 = col(np.linspace(0.47, 1.00, Ncool)) # decrease first number to have more white in the middle light-blue colors
#tmp2 = col(np.linspace(0.00, 0.51, Nwarm)) # increase second number to have more white in the middle light-yellow colors
#newcolors = np.append(tmp1[::-1,:],tmp2[::-1,:],axis=0)
#newcolors = col(np.linspace(0.47, 1.00,256))
#cmap_new = ListedColormap(newcolors)

levels = range(10,77)

#cax=fig.add_axes([0.52, 0.02, 0.43, 0.015]) # color bar
im0=plt.contourf(grd.x,grd.y,map_to_plot,extend='max',cmap=cmap_new2, norm=norm, levels=bounds)
plt.contourf(grd.x,grd.y,map_to_plot2,extend='max',cmap=cmap_new2, norm=norm, levels=bounds) #, alpha=0.7
#plt.contour(grd.x,grd.y,map_to_plot2,extend='max',colors='grey',zorder=15)
plt.contour(grd.x,grd.y,file_isf['ISF_mask'],levels,linewidths=0.7,colors='darkgrey',linestyles='solid',zorder=10) #.where(np.isnan(grounded_msk))
plt.contour(grd.x,grd.y,grounded_basin_kisf,levels,linewidths=0.7,colors='darkgrey',linestyles='solid',zorder=10)
plt.contour(grd.x,grd.y,grounded_msk,linewidths=0.7,colors='darkgrey',zorder=12)
plt.contour(grd.x,grd.y,icesheet_msk,linewidths=0.9,colors='black',zorder=15)

#if yy == 2100:
#    SMB_to_plot = nonvia_SMB_perc_mean.sel(scenario=scen)/100 
#    BMB_to_plot = nonvia_BMB_perc_mean.sel(scenario=scen)/100
#    calv_to_plot = calv_perc_mean.sel(scenario=scen)/100
#elif yy == 2300:
SMB_to_plot = nonvia_SMB_perc_mean_2300/100 
BMB_to_plot = nonvia_BMB_perc_mean_2300/100
calv_to_plot = calv_perc_mean_2300/100


center_dim_masked = center_dim.where(np.isfinite(BMB_to_plot))
ax.plot([center_dim_masked['center_x'].squeeze().values, center_dim_masked['center_x_oc'].squeeze().values], [center_dim_masked['center_y'].squeeze().values, center_dim_masked['center_y_oc'].squeeze().values], linestyle="-", color = 'k', linewidth = 0.5)


for kisf in tqdm(file_isf['Nisf']):
    
    pos_x = center_dim['center_x_oc'].sel(Nisf=kisf).squeeze().values
    pos_y = center_dim['center_y_oc'].sel(Nisf=kisf).squeeze().values
    
    
    prop_ABUMIP = ABUMIP_weighted_mean.sel(Nisf=kisf) / ABUMIP_weighted_mean.max().values
    prop_ref = flux_ref_weighted_mean.sel(Nisf=kisf) / ABUMIP_weighted_mean.max().values
    radius = np.sqrt(prop_ABUMIP/np.pi) * 4
    radius2 = np.sqrt(prop_ref/np.pi) * 4

    if (prop_ABUMIP > 0) & (prop_ABUMIP > prop_ref):
    
        
        if kisf.values in SMB_to_plot.Nisf.values:
            if np.isfinite(BMB_to_plot.sel(Nisf=kisf)):
                if SMB_to_plot.sel(Nisf=kisf) > 0:
                    size = [SMB_to_plot.sel(Nisf=kisf), BMB_to_plot.sel(Nisf=kisf), calv_to_plot.sel(Nisf=kisf)]
                else:
                    size = [-1*SMB_to_plot.sel(Nisf=kisf), BMB_to_plot.sel(Nisf=kisf) - SMB_to_plot.sel(Nisf=kisf), calv_to_plot.sel(Nisf=kisf)]
    
            else:
                size = [1]
                radius = SMB_to_plot.sel(Nisf=kisf)
    
                
        if kisf.values in SMB_to_plot.Nisf.values: 
            if SMB_to_plot.sel(Nisf=kisf) > 0:   
                
                #ax.plot([center_dim['center_x'].sel(Nisf=kisf).squeeze().values, center_dim['center_x_oc'].sel(Nisf=kisf).squeeze().values], 
                #        [center_dim['center_y'].sel(Nisf=kisf).squeeze().values, center_dim['center_y_oc'].sel(Nisf=kisf).squeeze().values], linestyle="-", color = 'k', linewidth = 0.5)
        
                ax_sub = inset_axes(ax, width=radius.values.item(), height=radius.values.item(), loc=10, 
                                   bbox_to_anchor=(pos_x, pos_y),
                                   bbox_transform=ax.transData, 
                                   borderpad=0)
                ax_sub.pie(np.array(size), colors=['orange','c','olive'], startangle=90)
                #ax_sub.set_aspect("equal")
            else:
                ax_sub = inset_axes(ax, width=radius.values.item(), height=radius.values.item(), loc=10, 
                                   bbox_to_anchor=(pos_x, pos_y),
                                   bbox_transform=ax.transData, 
                                   borderpad=0)
                wedges, _ = ax_sub.pie(np.array(size), colors=['c','c','olive'], hatch=['','',''], startangle=90) #//
                #ax_sub.set_aspect("equal") 
                pie_wedge = wedges[0]
                pie_wedge.set_edgecolor('orange')
                pie_wedge.set_linewidth(3)
                pie_wedge.set_facecolor('c')
                #pie_wedge.set_hatch('-') #///
        
        else:
                ax_sub = inset_axes(ax, width=radius.values.item()*np.nan, height=radius.values.item()*np.nan, loc=10, 
                                   bbox_to_anchor=(pos_x, pos_y),
                                   bbox_transform=ax.transData, 
                                   borderpad=0)
                ax_sub.pie(np.array(size), colors=['orange','c','olive'], startangle=90)
                ax_sub.set_aspect("equal")
                
        if np.isfinite(BMB_to_plot.sel(Nisf=kisf)):
    
            ax_sub2 = inset_axes(ax, width=radius2.values.item(), height=radius2.values.item(), loc=10, 
                           bbox_to_anchor=(pos_x, pos_y),
                           bbox_transform=ax.transData, 
                           borderpad=0)
            ax_sub2.pie(np.array([1]), colors=['lightgrey'], startangle=90)
    
            ax_sub2.set_aspect("equal")
            
    else:
        print(kisf.values, file_isf['isf_name'].sel(Nisf=kisf).values)

#ax2 = ax.twinx()
#ax2.scatter(center_dim['center_x_oc'], center_dim['center_y_oc'], s=flux_ref_weighted_mean, color='lightskyblue', zorder=20)

ax.set_title('Viability probability '+str(yy)+' ElmerIcegeo '+str(geoyear))
ax.set_axis_off()

#ax2 = fig.add_axes([0.90, 0.1, 0.03, 0.8])
#cb = mpl.colorbar.ColorbarBase(ax2, cmap=cmap_new, norm=norm,
#    spacing='proportional', ticks=np.arange(0.5,5), boundaries=bounds) #, format='%1i'
#cb.set_ticklabels(likelihood_list, rotation=90)

#plt.tight_layout()
fig.savefig(plot_path + 'map_2D_likelihood_'+str(yy)+'_withpiecharts_withproportionality_ElmerIcegeo'+str(geoyear)+'.png', dpi=300)

In [ ]:
SMB_to_plot

In [ ]:
scen = 'ssp585'
yy = 2300


fig, ax = plt.subplots(figsize=(8.25,8.25))
# MAR masks:
grd=file_isf['ISF_mask']
msk_ice = grd.where( (grd>0.) )* 0 + 1

# Basin masks:
basin=xr.open_dataset('/bettik/burgardc/DATA/SUMMER_PAPER/interim/Mask_Iceshelf_4km_IMBIE_withNisf.nc')

##########################################################################
# PLOT :

#----------
# Colorbar:
#cbar_range = np.arange(0.,4200.,200.)
#newcolors = col(np.linspace(0.47, 1.00,256))

#----------
# Defining colormap:

# moving the zero of colorbar
# NB: modify the Ncool to Nwarm ratio (total=256) to place zero as desired.
#Ncool=int(256*(-np.amin(cbar_range)/(np.amax(cbar_range)-np.amin(cbar_range))))
#Nwarm=256-Ncool
#col = cm.get_cmap('PuOr', 256)
#tmp1 = col(np.linspace(0.47, 1.00, Ncool)) # decrease first number to have more white in the middle light-blue colors
#tmp2 = col(np.linspace(0.00, 0.51, Nwarm)) # increase second number to have more white in the middle light-yellow colors
#newcolors = np.append(tmp1[::-1,:],tmp2[::-1,:],axis=0)
#newcolors = col(np.linspace(0.47, 1.00,256))
#cmap_new = ListedColormap(newcolors)

levels = range(10,77)

#cax=fig.add_axes([0.52, 0.02, 0.43, 0.015]) # color bar
im0=plt.contourf(grd.x,grd.y,map_to_plot,extend='max',cmap=cmap_new2, norm=norm, levels=bounds)
plt.contourf(grd.x,grd.y,map_to_plot2,extend='max',cmap=cmap_new2, norm=norm, levels=bounds) #, alpha=0.7
#plt.contour(grd.x,grd.y,map_to_plot2,extend='max',colors='grey',zorder=15)
#plt.contour(grd.x,grd.y,file_isf['ISF_mask'],levels,linewidths=0.7,colors='darkgrey',linestyles='solid',zorder=10) #.where(np.isnan(grounded_msk))
#plt.contour(grd.x,grd.y,grounded_basin_kisf,levels,linewidths=0.7,colors='darkgrey',linestyles='solid',zorder=10)
#plt.contour(grd.x,grd.y,grounded_msk,linewidths=0.7,colors='darkgrey',zorder=12)
#plt.contour(grd.x,grd.y,icesheet_msk,linewidths=0.9,colors='black',zorder=15)

#if yy == 2100:
#    SMB_to_plot = nonvia_SMB_perc_mean.sel(scenario=scen)/100 
#    BMB_to_plot = nonvia_BMB_perc_mean.sel(scenario=scen)/100
#    calv_to_plot = calv_perc_mean.sel(scenario=scen)/100
#elif yy == 2300:
SMB_to_plot = nonvia_SMB_perc_mean_2300.sel(scenario=scen)/100 
BMB_to_plot = nonvia_BMB_perc_mean_2300.sel(scenario=scen)/100
calv_to_plot = calv_perc_mean_2300.sel(scenario=scen)/100


center_dim_masked = center_dim.where(np.isfinite(BMB_to_plot))


kisf = 11

pos_x = center_dim['center_x_oc'].sel(Nisf=kisf).squeeze().values
pos_y = center_dim['center_y_oc'].sel(Nisf=kisf).squeeze().values


prop_100 = 100 / ABUMIP_weighted_mean.max().values
prop_500 = 500 / ABUMIP_weighted_mean.max().values
prop_1000 = 1000 / ABUMIP_weighted_mean.max().values
prop_4000 = 4000 / ABUMIP_weighted_mean.max().values


radius = np.sqrt(prop_100/np.pi) * 4
radius2 = np.sqrt(prop_500/np.pi) * 4
radius3 = np.sqrt(prop_1000/np.pi) * 4
radius4 = np.sqrt(prop_4000/np.pi) * 4

ax_sub = inset_axes(ax, width=radius4, height=radius4, loc=10, 
                   bbox_to_anchor=(pos_x, pos_y),
                   bbox_transform=ax.transData, 
                   borderpad=0)
ax_sub.pie(np.array([1.0]), colors=['white'], startangle=90, wedgeprops={'edgecolor': 'black'})

ax_sub = inset_axes(ax, width=radius3, height=radius3, loc=10, 
                   bbox_to_anchor=(pos_x, pos_y),
                   bbox_transform=ax.transData, 
                   borderpad=0)
ax_sub.pie(np.array([1.0]), colors=['white'], startangle=90, wedgeprops={'edgecolor': 'black'})

ax_sub = inset_axes(ax, width=radius2, height=radius2, loc=10, 
                   bbox_to_anchor=(pos_x, pos_y),
                   bbox_transform=ax.transData, 
                   borderpad=0)
ax_sub.pie(np.array([1.0]), colors=['white'], startangle=90, wedgeprops={'edgecolor': 'black'})

ax_sub = inset_axes(ax, width=radius, height=radius, loc=10, 
                   bbox_to_anchor=(pos_x, pos_y),
                   bbox_transform=ax.transData, 
                   borderpad=0)
ax_sub.pie(np.array([1.0]), colors=['white'], startangle=90, wedgeprops={'edgecolor': 'black'})

ax.set_axis_off()
fig.savefig(plot_path + 'piechart_size_100_500_1000_4000.pdf', dpi=300)

In [ ]:
BMB_to_plot.sel(Nisf=kisf)

In [ ]:
scen = 'ssp585'

fig, ax = plt.subplots(figsize=(8.25,8.25))
# MAR masks:
grd=file_isf['ISF_mask']
msk_ice = grd.where( (grd>0.) )* 0 + 1

# Basin masks:
basin=xr.open_dataset('/bettik/burgardc/DATA/SUMMER_PAPER/interim/Mask_Iceshelf_4km_IMBIE_withNisf.nc')

##########################################################################
# PLOT :

#----------
# Colorbar:
#cbar_range = np.arange(0.,4200.,200.)
#newcolors = col(np.linspace(0.47, 1.00,256))

#----------
# Defining colormap:

# moving the zero of colorbar
# NB: modify the Ncool to Nwarm ratio (total=256) to place zero as desired.
#Ncool=int(256*(-np.amin(cbar_range)/(np.amax(cbar_range)-np.amin(cbar_range))))
#Nwarm=256-Ncool
#col = cm.get_cmap('PuOr', 256)
#tmp1 = col(np.linspace(0.47, 1.00, Ncool)) # decrease first number to have more white in the middle light-blue colors
#tmp2 = col(np.linspace(0.00, 0.51, Nwarm)) # increase second number to have more white in the middle light-yellow colors
#newcolors = np.append(tmp1[::-1,:],tmp2[::-1,:],axis=0)
#newcolors = col(np.linspace(0.47, 1.00,256))
#cmap_new = ListedColormap(newcolors)

levels = range(10,77)

#cax=fig.add_axes([0.52, 0.02, 0.43, 0.015]) # color bar
im0=plt.contourf(grd.x,grd.y,map_to_plot,extend='max',cmap=cmap_new, norm=norm)
ax.contourf(grd.x,grd.y,map_to_plot2,extend='max',cmap=cmap_new, norm=norm, alpha=0.7)
#plt.contour(grd.x,grd.y,map_to_plot2,extend='max',colors='grey',zorder=15)
ax.contour(grd.x,grd.y,grounded_basin_kisf,levels,linewidths=0.5,colors='grey',zorder=0)
ax.contour(grd.x,grd.y,grounded_msk,linewidths=0.5,colors='black',zorder=0)
ax.contour(grd.x,grd.y,icesheet_msk,linewidths=0.1,colors='black',zorder=5)
ax.plot([center_dim['center_x'].squeeze().values, center_dim['center_x_oc'].squeeze().values], [center_dim['center_y'].squeeze().values, center_dim['center_y_oc'].squeeze().values], linestyle="-", color = 'k', linewidth = 0.5)

SMB_to_plot = nonvia_SMB_perc_mean_2300.sel(scenario=scen)/100 
BMB_to_plot = nonvia_BMB_perc_mean_2300.sel(scenario=scen)/100
calv_to_plot = calv_perc_mean_2300.sel(scenario=scen)/100

for kisf in tqdm(file_isf['Nisf']):
    pos_x = center_dim['center_x_oc'].sel(Nisf=kisf).squeeze().values
    pos_y = center_dim['center_y_oc'].sel(Nisf=kisf).squeeze().values
    
    prop_ABUMIP = ABUMIP_weighted_mean.sel(Nisf=kisf) / ABUMIP_weighted_mean.max().values
    prop_ref = flux_ref_weighted_mean.sel(Nisf=kisf) / ABUMIP_weighted_mean.max().values
    radius = np.sqrt(prop_ABUMIP/np.pi) * 4
    radius2 = np.sqrt(prop_ref/np.pi) * 4
    
    if kisf.values in SMB_to_plot.Nisf.values:
        if np.isfinite(BMB_to_plot.sel(Nisf=kisf)):
            if SMB_to_plot.sel(Nisf=kisf) > 0:
                size = [SMB_to_plot.sel(Nisf=kisf), BMB_to_plot.sel(Nisf=kisf), calv_to_plot.sel(Nisf=kisf)]
            else:
                size = [-1*SMB_to_plot.sel(Nisf=kisf), BMB_to_plot.sel(Nisf=kisf) - SMB_to_plot.sel(Nisf=kisf), calv_to_plot.sel(Nisf=kisf)]

        else:
            radius = SMB_to_plot.sel(Nisf=kisf)

            
    if kisf.values in SMB_to_plot.Nisf.values:    
        if SMB_to_plot.sel(Nisf=kisf) > 0:   
            ax_sub = inset_axes(ax, width=radius.values.item(), height=radius.values.item(), loc=10, 
                               bbox_to_anchor=(pos_x, pos_y),
                               bbox_transform=ax.transData, 
                               borderpad=0)
            ax_sub.pie(np.array(size), colors=['orange','blue','olive'], startangle=90)
            #ax_sub.set_aspect("equal")
        else:
            ax_sub = inset_axes(ax, width=radius.values.item(), height=radius.values.item(), loc=10, 
                               bbox_to_anchor=(pos_x, pos_y),
                               bbox_transform=ax.transData, 
                               borderpad=0)
            wedges, _ = ax_sub.pie(np.array(size), colors=['blue','blue','olive'], hatch=['//','',''], startangle=90) #
            #ax_sub.set_aspect("equal")
            pie_wedge = wedges[0]
            pie_wedge.set_edgecolor('orange')
            pie_wedge.set_facecolor('blue')
            pie_wedge.set_hatch('///')
    
    else:
            ax_sub = inset_axes(ax, width=radius.values.item()*np.nan, height=radius.values.item()*np.nan, loc=10, 
                               bbox_to_anchor=(pos_x, pos_y),
                               bbox_transform=ax.transData, 
                               borderpad=0)
            ax_sub.pie(np.array(size), colors=['orange','blue','olive'], startangle=90)
            #ax_sub.set_aspect("equal")
            
        

    ax_sub2 = inset_axes(ax, width=radius2.values.item(), height=radius2.values.item(), loc=10, 
                   bbox_to_anchor=(pos_x, pos_y),
                   bbox_transform=ax.transData, 
                   borderpad=0)
    ax_sub2.pie(np.array([1]), colors=['lightskyblue'], startangle=90)

    ax_sub2.set_aspect("equal")

#ax2 = ax.twinx()
#ax2.scatter(center_dim['center_x_oc'], center_dim['center_y_oc'], s=flux_ref_weighted_mean, color='lightskyblue', zorder=20)

ax.set_title('Viability probability '+str(yy)+' ElmerIcegeo '+str(geoyear)

#ax2 = fig.add_axes([0.90, 0.1, 0.03, 0.8])
#cb = mpl.colorbar.ColorbarBase(ax2, cmap=cmap_new, norm=norm,
#    spacing='proportional', ticks=np.arange(0.5,5), boundaries=bounds) #, format='%1i'
#cb.set_ticklabels(likelihood_list, rotation=90)

#plt.tight_layout()
fig.savefig(plot_path + 'map_2D_likelihood_'+str(yy)+'_withpiecharts_withproportionality_ElmerIcegeo'+str(geoyear)+'.png', dpi=300)

In [ ]:
for kisf in tqdm(file_isf['Nisf']):
    print(kisf.values,file_isf['isf_name'].sel(Nisf=kisf).values)

In [ ]:
fig,ax = plt.subplots()
ax.set_aspect('equal')
p1,t1 = plt.pie([50,50], center=(0,0))
p2,t2 = plt.pie([1,1,1,1], center=(1.2,0)) # this pie-chart is over the first one

[p.set_zorder(-1) for p in p2]

In [ ]:
plt.scatter(center_dim['center_x_oc'],center_dim['center_y_oc'], s= flux_ref_weighted_mean, color='lightskyblue')

In [ ]:
radius_outer

In [ ]:
np.array(radius.values)

In [ ]:
    ax_sub= inset_axes(ax, width=width, height=width, loc=10, 
                       bbox_to_anchor=(ilon, ilat),
                       bbox_transform=ax.transData, 
                       borderpad=0)
    wedges,texts= ax_sub.pie(data)

    ax_sub.set_aspect("equal")

In [ ]:
(flux_ref_weighted_mean.sel(Nisf=kisf))/ABUMIP_weighted_mean.sel(Nisf=kisf)

In [ ]:
(ABUMIP_weighted_mean.sel(Nisf=kisf) - flux_ref_weighted_mean.sel(Nisf=kisf))/ABUMIP_weighted_mean.sel(Nisf=kisf)

In [ ]:
flux_ref_weighted_mean

In [ ]:
ABUMIP_weighted_mean

In [ ]:
(ABUMIP_weighted_mean.sel(Nisf=kisf)/ABUMIP_weighted_mean.max()).values

In [ ]:
kisf = 10
a = plt.pie(
    [(flux_ref_weighted_mean.sel(Nisf=kisf))/ABUMIP_weighted_mean.sel(Nisf=kisf), (ABUMIP_weighted_mean.sel(Nisf=kisf) - flux_ref_weighted_mean.sel(Nisf=kisf))/ABUMIP_weighted_mean.sel(Nisf=kisf)],
    colors = ['lightblue', 'deepskyblue' ],
    # wedgeprops={'alpha':1},
    radius = (ABUMIP_weighted_mean.sel(Nisf=kisf)/ABUMIP_weighted_mean.max()).values + 0.5) 


In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=1,figsize=(18.0,18.0))
#im0=axs.contourf(grd.x,grd.y,via_to_plot,extend='max',cmap=cmap_new, norm=norm)
axs.contour(grd.x,grd.y,grounded_msk,linewidths=0.5,colors='black',zorder=10)
axs.contour(grd.x,grd.y,icesheet_msk,linewidths=0.5,colors='black',zorder=15)
axs.scatter(center_dim['center_x'],center_dim['center_y'])
axs.scatter(center_dim['center_x_oc'],center_dim['center_y_oc'])